In [1]:
"""Strategy module as Trading Strategy 0.1 module format.

To run a backtest::

    trade-executor \
        start \
        --strategy-file=enzyme-polygon-eth-usdc-sls.py \
        --state-fileenzyme-polygon-eth-usdc-sls-backtest.json \
        --asset-management-mode=backtest \
        --backtest-start=2023-01-01 \
        --backtest-end=2023-04-01


"""

import pandas as pd
import numpy as np
import pandas_ta as ta
import datetime


from tradingstrategy.client import Client


# This is added to improve the image resolution of matplotlib charts generated by quantstats library
%config InlineBackend.figure_format = 'svg'

from tradeexecutor.visual.single_pair import visualise_single_pair, visualise_single_pair_positions_with_duration_and_slippage
from tradingstrategy.charting.candle_chart import VolumeBarMode


from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse
from tradeexecutor.strategy.trading_strategy_universe import load_partial_data
from tradeexecutor.strategy.execution_context import ExecutionContext
from tradeexecutor.strategy.execution_context import ExecutionMode
from tradeexecutor.strategy.universe_model import UniverseOptions
from tradeexecutor.strategy.execution_context import ExecutionMode
from tradeexecutor.visual.equity_curve import calculate_equity_curve, calculate_returns
from tradeexecutor.visual.equity_curve import visualise_equity_curve
from tradeexecutor.analysis.trade_analyser import build_trade_analysis
import logging
from tradeexecutor.analysis.advanced_metrics import (
    visualise_advanced_metrics,
    AdvancedMetricsMode,
)


from tradeexecutor.state.visualisation import PlotKind
from tradeexecutor.strategy.cycle import CycleDuration
from tradeexecutor.strategy.default_routing_options import TradeRouting
from tradeexecutor.strategy.execution_context import ExecutionContext
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager
from tradeexecutor.strategy.reserve_currency import ReserveCurrency
from tradeexecutor.strategy.strategy_type import StrategyType
from tradeexecutor.strategy.trading_strategy_universe import (
    load_pair_data_for_single_exchange,
    TradingStrategyUniverse,
)

from tradeexecutor.strategy.trading_strategy_universe import translate_trading_pair
from tradeexecutor.strategy.trading_strategy_universe import load_partial_data
from tradeexecutor.strategy.universe_model import UniverseOptions
from tradingstrategy.client import Client
from tradingstrategy.timebucket import TimeBucket
from tradingstrategy.chain import ChainId
from tradeexecutor.backtest.backtest_runner import run_backtest_inline

# ================================================================================================= #
#                                      GLOBAL Variables                                             #
# ================================================================================================= #
TRADING_STRATEGY_ENGINE_VERSION = "0.1"

# What kind of strategy we are running.
# This tells we are going to use
# NOTE: this setting has currently no effect
TRADING_STRATEGY_TYPE = StrategyType.managed_positions

# We trade on Polygon
CHAIN_ID = ChainId.polygon

# How often the strategy performs the decide_trades cycle.
# TRADING_STRATEGY_CYCLE = CycleDuration.cycle_1d
# CANDLE_TIME_BUCKET = TimeBucket.d1
TRADING_STRATEGY_CYCLE = CycleDuration.cycle_1h
CANDLE_TIME_BUCKET = TimeBucket.h1

# Strategy keeps its cash in USDC
RESERVE_CURRENCY = ReserveCurrency.usdc

# Which trading pair we are backtesting on
# (Might be different from the live trading pair)
TRADING_PAIRS = [
    (ChainId.polygon, "quickswap", "WETH", "USDC", 0.003),
    # (ChainId.polygon, "quickswap", "WMATIC", "USDC", 0.003),
    # (ChainId.polygon, "quickswap", "GHST", "USDC", 0.003),
    # (ChainId.polygon, "quickswap", "CHP", "USDT", 0.003),
    # (ChainId.polygon, "quickswap", "IXT", "USDT", 0.003),
    # (ChainId.polygon, "quickswap", "WLD", "USDC", 0.003),
    # (ChainId.polygon, "quickswap", "ICE", "USDC", 0.003),
    # (ChainId.polygon, "quickswap", "TEL", "USDC", 0.003)
]

EXAMINED_PAIR = TRADING_PAIRS[0]

# The duration of the backtesting period
START_AT = datetime.datetime(2013, 6, 1)
END_AT = datetime.datetime(2023, 7, 31)

# Start with 10,000 USD
INITIAL_DEPOSIT = 10_000
RESERVE_CURRENCY = ReserveCurrency.usdc
TRADE_ROUTING = TradeRouting.uniswap_v3_usdc_poly


# ================================================================================================= #
#                                     Different Methods                                             #
# ================================================================================================= #


def calculate_size(state, close):
    cash = state.portfolio.get_current_cash()
    return cash * 0.99 / len(TRADING_PAIRS)

current_sl = np.inf

def plot():
    examined_pair = universe.get_pair_by_human_description(EXAMINED_PAIR)

    figure = visualise_single_pair(
        state,
        universe.universe.candles,
        pair_id=examined_pair.internal_id,
        start_at=START_AT,
        end_at=END_AT,
        volume_axis_name="Volume (USD)",
        height=1000,
    )

    figure.show()

def stats():
    trade_count = len(list(state.portfolio.get_all_trades()))
    print(f"Backtesting completed, backtested strategy made {trade_count} trades")


    equity = calculate_equity_curve(state)
    returns = calculate_returns(equity)
    metrics = visualise_advanced_metrics(returns, mode=AdvancedMetricsMode.full)

    returns = metrics.loc["Cumulative Return"]
    dd = metrics.loc["Max Drawdown"]
    # with pd.option_context("display.max_row", None):
    #     display(metrics)
    print("==========")
    # print(f"Total return: {returns['Strategy']}")
    print(f"Max Drawdown: {dd['Strategy']}")


    analysis = build_trade_analysis(state.portfolio)

    summary = analysis.calculate_summary_statistics()

    with pd.option_context("display.max_row", None):
        display(summary.to_dataframe())

def decide_trades(timestamp, universe, state, pricing_model, cycle_debug_data):
    # Create a position manager helper class that allows us easily to create
    # opening/closing trades for different positions
    position_manager = PositionManager(timestamp, universe, state, pricing_model)

    # Watch out for the inclusive range and include and avoid peeking in the future
    adjusted_timestamp = timestamp - pd.Timedelta(seconds=1)

    # trading_strategy_cycle = CycleDuration.cycle_7d
    # momentum_lookback_period = datetime.timedelta(days=1)
    # start = (
    #     adjusted_timestamp - momentum_lookback_period - datetime.timedelta(seconds=1)
    # )
    # end = adjusted_timestamp
    # candle_universe = universe.candles
    # candle_data = candle_universe.iterate_samples_by_pair_range(start, end)

    LOOKBACK_CANDLE_COUNT = 201
    start = timestamp - (
        LOOKBACK_CANDLE_COUNT * CANDLE_TIME_BUCKET.to_pandas_timedelta()
    )
    end = timestamp - CANDLE_TIME_BUCKET.to_pandas_timedelta()
    candle_data = universe.candles.iterate_samples_by_pair_range(start, end)

    # The pair we are trading
    trades = []

    # Iterate over all candles for all pairs in this timestamp (ts)
    for pair_id, candles in candle_data:
        # We should have candles for range start - end,
        # where end is the current strategy cycle timestamp
        # and start is one week before end.
        # Because of sparse data we may have 0, 1 or 2 candles
        # Convert raw trading pair data to strategy execution format
        pair_data = universe.pairs.get_pair_by_id(pair_id)
        pair = translate_trading_pair(pair_data)

        # We have data for open, high, close, etc.
        # We only operate using candle close values in this strategy.

        supertrend = ta.supertrend(
            high=candles["high"],
            low=candles["low"],
            close=candles["close"],
            period=7,
            multiplier=3,
        )

        sma = ta.sma(candles["close"], length=200)

        if supertrend is None:
            continue
        if sma is None:
            continue
        supertrend = supertrend["SUPERT_7_3.0"]
        if np.isnan(supertrend.iloc[-1]):
            continue

        entry = (
            candles["close"].iloc[-1] > sma.iloc[-1]
            and candles["close"].iloc[-1] > supertrend.iloc[-1]
        )
        exit = candles["close"].iloc[-1] < supertrend.iloc[-1]

        # Create a position manager helper class that allows us easily to create
        # opening/closing trades for different positions
        current_price = candles["close"].iloc[-1]
        position_manager = PositionManager(timestamp, universe, state, pricing_model)
        buy_amount = calculate_size(state, current_price)

        position_for_pair = position_manager.get_current_position_for_pair(pair)

        if not position_for_pair:
            if entry:
                trades += position_manager.open_1x_long(pair, buy_amount)
        else:
            if exit:
                symbol = pair.base.token_symbol
                # print(candles["close"].iloc[-1], sma.iloc[-1], supertrend.iloc[-1])
                # print("exiting")
                trades += position_manager.close_position(position_for_pair)

        visualisation = state.visualisation
        symbol = pair.base.token_symbol
        if symbol == EXAMINED_PAIR[2]:
            # print("-", supertrend.iloc[-1])
            # print(supertrend)
            # print(type(supertrend))

            visualisation.plot_indicator(
                timestamp,
                f"{symbol} Supertrend",
                PlotKind.technical_indicator_on_price,
                supertrend.iloc[-1],
            )
            visualisation.plot_indicator(
                timestamp,
                f"{symbol} SMA",
                PlotKind.technical_indicator_on_price,
                sma.iloc[-1],
            )

    return trades


def create_trading_universe(
    ts: datetime.datetime,
    client: Client,
    execution_context: ExecutionContext,
    universe_options: UniverseOptions,
) -> TradingStrategyUniverse:
    assert (
        not execution_context.mode.is_live_trading()
    ), f"Only strategy backtesting supported, got {execution_context.mode}"

    # Load data for our trading pair whitelist
    dataset = load_partial_data(
        client=client,
        time_bucket=TRADING_STRATEGY_CYCLE.to_timebucket(),
        pairs=TRADING_PAIRS,
        execution_context=execution_context,
        universe_options=universe_options,
        liquidity=False,
        start_at=START_AT,
        end_at=END_AT,
    )

    # Filter down the dataset to the pairs we specified
    universe = TradingStrategyUniverse.create_multichain_universe_by_pair_descriptions(
        dataset,
        TRADING_PAIRS,
        reserve_token_symbol="USDC",  # Pick any USDC - does not matter as we do not route
    )

    return universe


client = Client.create_jupyter_client()
universe = create_trading_universe(
    datetime.datetime.utcnow(),
    client,
    ExecutionContext(mode=ExecutionMode.backtesting),
    universe_options=UniverseOptions(),
)

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /home/alpha/.tradingstrategy


In [2]:
import time

start = time.time()
state, _, debug_dump = run_backtest_inline(
    name="BLS",
    start_at=START_AT,
    end_at=END_AT,
    client=client,
    cycle_duration=TRADING_STRATEGY_CYCLE,
    decide_trades=decide_trades,
    universe=universe,
    initial_deposit=INITIAL_DEPOSIT,
    reserve_currency=RESERVE_CURRENCY,
    trade_routing=TRADE_ROUTING,
    log_level=logging.WARNING,
)
end = time.time()
plot()
stats()
print(end - start)

Backtesting completed, backtested strategy made 14 trades
Max Drawdown: -65.59%
                                                  0
Trading period length                      677 days
Return %                                     86.21%
Annualised return %                          46.48%
Cash at start                            $10,000.00
Value at end                             $18,620.57
Trade volume                            $272,358.93
Position win percent                         57.14%
Total positions                                   7
Won positions                                     4
Lost positions                                    3
Stop losses triggered                             0
Stop loss % of all                            0.00%
Stop loss % of lost                           0.00%
Winning stop losses                               0
Winning stop losses percent                       -
Losing stop losses                                0
Losing stop losses percent          